# Loading libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
        
from scipy.stats import skew,norm,zscore
from scipy.signal import periodogram

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Importing the dataset

In [2]:
orig_holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=['date'])
orig_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", parse_dates=['date'])
orig_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
orig_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv", parse_dates=['date'])

orig_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'])
orig_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'])

# Setting main dates

In [3]:
date = {
    'date_start_train': '2013-01-01',
    'date_end_train': '2017-08-15',
    'date_start_test': '2017-08-16',
    'date_end_test': '2017-08-31',
    'date_start_fore': '2016-06-01'
}

In [4]:
diff_train = (pd.Timestamp(date['date_end_train']) - pd.Timestamp(date['date_start_fore'])).days
diff_test = (pd.Timestamp(date['date_end_test']) - pd.Timestamp(date['date_start_fore'])).days

# Stores

In [5]:
def store_func (orig_df):
    
    df = orig_df.copy()
    
    # Adding features to orig_stores
    df['uniquestore'] = df.city.apply(lambda x: 0 if x in ['Quito', 'Guayaquil', 'Santo Domingo', 'Cuenca', 'Manta', 'Machala', 'Latacunga', 'Ambato'] else 1)
    df['newstore'] = df.store_nbr.apply(lambda x: 1 if x in [19, 20, 21, 28, 35, 41, 51, 52] else 0)
        
    # Merging orig_stores, orig_test and orig_train
    df = pd.concat([orig_train, orig_test], axis=0).merge(df, on=['store_nbr'], how='left')
    df = df.rename(columns={'type' : 'store'}) 

    return df

In [6]:
final_df = store_func(orig_stores)

# Events

In [7]:
def holiday_func (orig_df):
    
    df = orig_df.copy()
    
    # Non-transferred events
    df.loc[297, 'transferred'] = df.loc[297, 'transferred'] = False
    df = df.query("transferred!=True")
    
    # Removing duplicates
    df = df.drop(index=orig_holidays_events[orig_holidays_events[['date', 'locale_name']].duplicated()].index.values)

    # Adding event type
    df.loc[df.type=='Event', 'type'] = df.description.apply(lambda x: x[0:7])
     
    # Merging orig_holidays_events and final_df
    nat_df = df.query("locale=='National'")
    loc_df = df.query("locale=='Local'")
    reg_df = df.query("locale=='Regional'")
    
    df = final_df.merge(nat_df, left_on=['date'], right_on=['date'], how='left')
    df = df.merge(loc_df, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
    df = df.merge(reg_df, left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left')
   
    # Adding New Year
    df['firstday'] = df.description_x.apply(lambda x: 1 if x=='Primer dia del ano' else 0)

    # Matching event and store
    df = df.drop(columns=['locale_x', 'locale_name_x', 'description_x', 'transferred_x',
                          'locale_y', 'locale_name_y', 'description_y', 'transferred_y',
                          'locale', 'locale_name', 'description', 'transferred'])
    df.loc[~df.type_x.isnull(), 'event_type'] = df.type_x.apply(lambda x: x)
    df.loc[~df.type_y.isnull(), 'event_type'] = df.type_y.apply(lambda x: x)
    df.loc[~df.type.isnull(), 'event_type'] = df.type.apply(lambda x: x)
    df.loc[df.event_type.isnull(), 'event_type'] = df.event_type.apply(lambda x: 'norm')
    df = df.drop(columns=['type_x', 'type_y', 'type'])

    df['isevent'] = df.event_type.apply(lambda x: 'y' if x!='norm' else 'n')

    # Adding Easter
    df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'isevent'] = df.isevent.apply(lambda x: 'y')
    df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'event_type'] = df.event_type.apply(lambda x: 'Holiday')

    # Adding closure days
    df['isclosed'] = df.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x: 1 if x.sum()==0 else 0)    
    df.loc[(df.date.dt.year==2017) & (df.date.dt.month==8) & (df.date.dt.day>=16) , 'isclosed'] = df.isclosed.apply(lambda x: 0)    
    df.loc[df.date.isin(['2017-01-01']), 'isevent'] = df.isevent.apply(lambda x: 'n')
  
    return df

In [8]:
final_df = holiday_func(orig_holidays_events)

# Oil

In [9]:
orig_oil.set_index('date').resample("D").mean().isnull().sum()

dcoilwtico    529
dtype: int64

In [10]:
def oil_func (orig_df):
    
    df = orig_df.copy()
    
    # Adding missing values
    df = df.set_index('date').resample("D").mean().interpolate(limit_direction='backward').reset_index()
    
    for i in [1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 90]:
        df['lagoil_' + str(i) + '_dcoilwtico'] = df['dcoilwtico'].shift(i)
    
    df['oil_week_avg'] = df['dcoilwtico'].rolling(7).mean()
    df['oil_2weeks_avg'] = df['dcoilwtico'].rolling(14).mean()
    df['oil_month_avg'] = df['dcoilwtico'].rolling(30).mean()

    df.dropna(inplace = True)
    
    # Merging orig_oil and final_df
    df = final_df.merge(df, on=['date'], how='left')
    
    return df

In [11]:
final_df = oil_func(orig_oil)

# Transactions

In [12]:
def transactions_func (orig_df):
    
    df = orig_df.copy()
    
    # Merging orig_transactions and final_df
    df = final_df.merge(df, on=['date', 'store_nbr'], how='left')
    
    # Filling missing values
    df.loc[(df.transactions.isnull()) & (df.isclosed==1), 'transactions'] = df.transactions.apply(lambda x: 0)
    group_df = df.groupby(by=['store_nbr', 'date']).transactions.first().reset_index()
    group_df['avg_tra'] = group_df.transactions.rolling(15, min_periods=10).mean()
    group_df['16_tra'] = group_df.transactions.shift(16)
    group_df['21_tra'] = group_df.transactions.shift(21)
    group_df['30_tra'] = group_df.transactions.shift(30)
    group_df['60_tra'] = group_df.transactions.shift(60)
    group_df.drop(columns='transactions', inplace=True)
    df = df.merge(group_df, on=['date', 'store_nbr'], how='left')
    df.loc[(df.transactions.isnull()) & (df.isclosed==0), 'transactions'] = df.avg_tra
    df.drop(columns='avg_tra', inplace=True)
    df.loc[(df.date.dt.year==2017) & (df.date.dt.month==8) & (df.date.dt.day>=16) , 'transactions'] = df.transactions.apply(lambda x: None)    

    df['tot_store_day_onprom'] = df.groupby(by=['date', 'store_nbr']).onpromotion.transform(lambda x: x.sum())

    return df

In [13]:
final_df = transactions_func(orig_transactions)

In [14]:
del orig_train
del orig_test
del orig_stores
del orig_holidays_events
del orig_oil
del orig_transactions

# Final dataframe

In [15]:
final_df = final_df.set_index('date').loc[date['date_start_fore']:,:]

# Support functions

In [16]:
def split_func (orig_df, X, y, end_date, test_size):
    
    # Splitting train and test
    idx_train, idx_test = train_test_split(orig_df.index, test_size=test_size, shuffle=False)
    X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
    y_train, y_test = y.loc[idx_train], y.loc[idx_test]
    
    return X_train, y_train, X_test, y_test

In [17]:
def my_split_func(df, train_start_date='2013-01-01', train_end_date='2017-08-30',
               val_start_date='2017-09-01', val_end_date='2020-01-01'):
    train_start_date = (pd.to_datetime(train_start_date) - pd.to_datetime('2013-01-01')).days
    train_end_date = (pd.to_datetime(train_end_date) - pd.to_datetime('2013-01-01')).days
    val_start_date = (pd.to_datetime(val_start_date) - pd.to_datetime('2013-01-01')).days
    val_end_date = (pd.to_datetime(val_end_date) - pd.to_datetime('2013-01-01')).days
    
    train = df[(df['days_from_2013'] >= train_start_date) & (df['days_from_2013'] <= train_end_date)]
    val = df[(df['days_from_2013'] >= val_start_date) & (df['days_from_2013'] <= val_end_date)]
    return [train.drop(columns=['sales']), val.drop(columns=['sales']), train['sales'], val['sales']]

In [18]:
def get_weights_distribution(tp, dates):
    if tp == 1:
        return np.ones(dates.shape)
    if tp == 2:
        return np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(dates)).days) / 100)
    if tp == 3:
        return np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(dates)).days) / 200)
    if tp == 4:
        return np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(dates)).days) / 300)
    if tp == 5:
        return np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(dates)).days) / 400)

In [19]:
def tags_to_dict():
    tags = {
     'AUTOMOTIVE': [4, 7, 30, 10, 'family'],
     'BABY CARE':  [-8, 2, 25, 5, 'family'],
     'BEAUTY': [-8, 7, 25, 5, 'other'],
     'BEVERAGES': [0, 0, 40, 40, 'food'],
     'BOOKS': [0, 0, 55, 15, 'other'],
     'BREAD/BAKERY': [-3, 0, 30, 30, 'food'],
     'CELEBRATION': [-5, 5, 50, 20, 'family'],
     'CLEANING': [-8, 3, 40, 20, 'food'],
     'DAIRY': [-4, 0, 40, 40, 'food'],
     'DELI': [3, 6, 40, 20, 'food'],
     'EGGS': [-4, -5, 40, 20, 'food'],
     'FROZEN FOODS': [-4, -3, 40, 20, 'food'],
     'GROCERY I': [-4, 3, 40, 20, 'food'],
     'GROCERY II': [-4, 3, 40, 20, 'food'],
     'HARDWARE': [10, 10, 30, 20, 'other'],
     'HOME AND KITCHEN I': [-10, 4, 40, 20, 'family'],
     'HOME AND KITCHEN II': [-10, 4, 40, 20, 'family'],
     'HOME APPLIANCES': [0, 4, 40, 20, 'family'],
     'HOME CARE': [-10, 4, 40, 20, 'family'],
     'LADIESWEAR': [-10, 4, 40, 20, 'other'],
     'LAWN AND GARDEN': [-10, 4, 40, 20, 'family'],
     'LINGERIE': [-10, 4, 40, 2, 'other'],
     'LIQUOR,WINE,BEER': [4, 8, 40, 20, 'food'],
     'MAGAZINES': [-6, -7, 50, 20, 'other'],
     'MEATS': [-4, 5, 40, 20, 'food'],
     'PERSONAL CARE': [-5, 5, 40, 20, 'family'],
     'PET SUPPLIES': [-5, 0, 40, 20, 'family'],
     'PLAYERS AND ELECTRONICS': [5, 5, 25, 10, 'other'],
     'POULTRY': [-7, -4, 40, 20, 'food'],
     'PREPARED FOODS': [0, 6, 30, 10, 'food'],
     'PRODUCE': [0, 0, 40, 40, 'other'],
     'SCHOOL AND OFFICE SUPPLIES': [3, 3, 25, 15, 'family'],
     'SEAFOOD': [-5, 8, 40, 20, 'food']
    }
    
    sex_dict = {}
    luxury_dict = {}
    age_mean_dict = {}
    age_var_dict = {}
    type_dict = {}
    for i in tags.keys():
        sex_dict[i] = tags[i][0]
        luxury_dict[i] = tags[i][1]
        age_mean_dict[i] = tags[i][2]
        age_var_dict[i] = tags[i][3]
        type_dict[i] = tags[i][4]
    return [sex_dict, luxury_dict, age_mean_dict, age_var_dict, type_dict]

In [20]:
def get_oil_dict(oil):
    # estimate price of gaps (market don't work on weekends and holidays)
    price_estim = [-1] * (oil['days_from_2013'][oil.shape[0] - 1] + 1)
    price_estim[0] = 93.14
    for i in range(1, oil.shape[0]):
        price_estim[oil['days_from_2013'][i]] = oil['dcoilwtico'][i]

    for i in range (len(price_estim)):
        if price_estim[i] == -1 or math.isnan(price_estim[i]):
            tj = -1
            for j in range(i + 1, len(price_estim)):
                if price_estim[j] != -1 and (not math.isnan(price_estim[j])):
                    tj = j
                    break

            for j in range(i, tj):
                price_estim[j] = ((tj - j) * price_estim[i - 1] + (j - i) * price_estim[tj]) / (tj - i)

            i = tj

    oil_dict = dict(zip(np.arange(len(price_estim)), price_estim))
    return oil_dict

# Last stripes

In [21]:
def add_features (orig_df):
    
    df = orig_df.copy()
        
    # Time features
    df['year'] = df.index.year.astype('int')
    df['quarter'] = df.index.quarter.astype('int')
    df['month'] = df.index.month.astype('int')
    df['day'] = df.index.day.astype('int')
    df['dayofweek'] = df.index.day_of_week.astype('int')
    df['weekofyear'] = df.index.week.astype('int')
    df['isweekend'] = df.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)
    df['startschool'] = df.month.apply(lambda x: 1 if x in (4,5,8,9) else 0)
    
    df['daysinmonth'] = df.index.days_in_month.astype('int')

    # Dummy features
    df = pd.get_dummies(df, columns=['year'], drop_first=True)
    df = pd.get_dummies(df, columns=['quarter'], drop_first=True)
    df = pd.get_dummies(df, columns=['dayofweek'], drop_first=True)
    df = pd.get_dummies(df, columns=['store'], drop_first=True)
    df = pd.get_dummies(df, columns=['event_type'], drop_first=True)
    df = pd.get_dummies(df, columns=['isevent'], drop_first=True)
    df = pd.get_dummies(df, columns=['state'], drop_first=True)

    # DeterministicProcess
    fourierA = CalendarFourier(freq='A', order=5)
    fourierM = CalendarFourier(freq='M', order=2)
    fourierW = CalendarFourier(freq='W', order=4)

    dp = DeterministicProcess(index=df.index,
                          order=1,
                          seasonal=False,
                          constant=False,
                          additional_terms=[fourierA, fourierM, fourierW],
                          drop=True)
    dp_df = dp.in_sample()
    df = pd.concat([df, dp_df], axis=1)
    
    # Outliers
    df['outliers'] = df.sales.apply(lambda x: 1 if x>30000 else 0)
    
    df.drop(columns=['daysinmonth', 'month', 'city'], inplace=True)
    
    return df

In [22]:
import math

def add_my(df):
    # read
    train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
    oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
    trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
    
    # add 'days_from_2013' for easy shifting
    df['days_from_2013'] = (pd.to_datetime(df.index.get_level_values(2)) - pd.to_datetime('2013-01-01')).days
    train['days_from_2013'] = (pd.to_datetime(train['date']) - pd.to_datetime('2013-01-01')).dt.days
    oil['days_from_2013'] = (pd.to_datetime(oil['date']) - pd.to_datetime('2013-01-01')).dt.days
    trans['days_from_2013'] = (pd.to_datetime(trans['date']) - pd.to_datetime('2013-01-01')).dt.days
    
    # groupby features
    gr_day = train.groupby('days_from_2013')['sales'].mean()
    gr_store = train.groupby('store_nbr')['sales'].mean()
    gr_family = train.groupby('family')['sales'].mean()

    days = [16, 18, 20, 21, 25, 28, 30, 35, 42, 60, 90, 120, 180, 365]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['days_lagged' + str(i)] = df['days_' + str(i)].map(gr_day).fillna(0)
        df = df.drop(columns=['days_' + str(i)])

    df['store_gb'] = df.index.get_level_values(0).map(gr_store)
    df['family_gb'] = df.index.get_level_values(1).map(gr_family)
    
    oil_dict = get_oil_dict(oil)

    # lagged oil
    days = [0, 1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 90, 120, 180, 360]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['oil_lagged' + str(i)] = df['days_' + str(i)].map(oil_dict)
        df = df.drop(columns=['days_' + str(i)])
        
    # lagged transactions
    # # fill trans dict
    trans_dict = {}
    for ii in range(trans.shape[0]):
        i = trans.loc[ii]
        trans_dict[tuple([i['store_nbr'], i['days_from_2013']])] = i['transactions']
    
    def transaction_get_value(a, b):
        try:
            return trans_dict[tuple([a, (pd.to_datetime(b) - pd.to_datetime('2013-01-01').dt.days)])]
        except:
            return 0

    days = [16, 18, 20, 21, 25, 28, 30, 35, 42, 60, 90, 120, 180, 365]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['oil_lagged' + str(i)] = df['days_' + str(i)].map(oil_dict)
        df['trans_lagged' + str(i)] = [transaction_get_value(*a) for a in tuple(zip(df.index.get_level_values(0),
                                                                        df.index.get_level_values(2)))]
        df = df.drop(columns=['days_' + str(i)])

    sex_dict, luxury_dict, age_mean_dict, age_var_dict, type_dict = tags_to_dict()
    df['tag_sex'] = df.index.get_level_values(1).map(sex_dict)
    df['tag_luxury'] = df.index.get_level_values(1).map(luxury_dict)
    df['tag_age_mean'] = df.index.get_level_values(1).map(age_mean_dict)
    df['tag_age_var'] = df.index.get_level_values(1).map(age_var_dict)
    df['tag_type'] = df.index.get_level_values(1).map(type_dict)
    df = pd.get_dummies(df, columns=['tag_type'])
    
    df['tag_age_min'] = df['tag_age_mean'] - df['tag_age_var']
    df['tag_age_max'] = df['tag_age_mean'] + df['tag_age_var']
    return df

In [23]:
df = add_features(final_df).loc[:date['date_end_test'],:].reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()

df['16_tra'] = df['16_tra'].fillna(0)
df['21_tra'] = df['21_tra'].fillna(0)
df['30_tra'] = df['30_tra'].fillna(0)
df['60_tra'] = df['60_tra'].fillna(0)
df = add_my(df)
display(df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  # This is added back by InteractiveShellApp.init_path()


id  sales  onpromotion  cluster  \
store_nbr family     date                                               
1         AUTOMOTIVE 2016-06-01  2216808    3.0            0       13   
                     2016-06-02  2218590    1.0            0       13   
                     2016-06-03  2220372    4.0            0       13   
                     2016-06-04  2222154    9.0            0       13   
                     2016-06-05  2223936    2.0            0       13   
...                                  ...    ...          ...      ...   
54        SEAFOOD    2017-08-27  3022139    NaN            0        3   
                     2017-08-28  3023921    NaN            0        3   
                     2017-08-29  3025703    NaN            0        3   
                     2017-08-30  3027485    NaN            0        3   
                     2017-08-31  3029267    NaN            0        3   

                                 uniquestore  newstore  firstday  isclosed  \
store_nbr family     date                                                    
1         AUTOMOTIVE 2016-06-01            0         0         0         0   
                     2016-06-02            0         0         0         0   
                     2016-06-03            0         0         0         0   
                     2016-06-04            0         0         0         0   
                     2016-06-05            0         0         0         0   
...                                      ...       ...       ...       ...   
54        SEAFOOD    2017-08-27            1         0         0         0   
                     2017-08-28            1         0         0         0   
                     2017-08-29            1         0         0         0   
                     2017-08-30            1         0         0         0   
                     2017-08-31            1         0         0         0   

                                 dcoilwtico  lagoil_1_dcoilwtico  ...  \
store_nbr family     date                                         ...   
1         AUTOMOTIVE 2016-06-01   49.070000            49.100000  ...   
                     2016-06-02   49.140000            49.070000  ...   
                     2016-06-03   48.690000            49.140000  ...   
                     2016-06-04   49.030000            48.690000  ...   
                     2016-06-05   49.370000            49.030000  ...   
...                                     ...                  ...  ...   
54        SEAFOOD    2017-08-27   46.816667            47.233333  ...   
                     2017-08-28   46.400000            46.816667  ...   
                     2017-08-29   46.460000            46.400000  ...   
                     2017-08-30   45.960000            46.460000  ...   
                     2017-08-31   47.260000            45.960000  ...   

                                 trans_lagged365  tag_sex  tag_luxury  \
store_nbr family     date                                               
1         AUTOMOTIVE 2016-06-01                0        4           7   
                     2016-06-02                0        4           7   
                     2016-06-03                0        4           7   
                     2016-06-04                0        4           7   
                     2016-06-05                0        4           7   
...                                          ...      ...         ...   
54        SEAFOOD    2017-08-27                0       -5           8   
                     2017-08-28                0       -5           8   
                     2017-08-29                0       -5           8   
                     2017-08-30                0       -5           8   
                     2017-08-31                0       -5           8   

                                 tag_age_mean  tag_age_var  tag_type_family  \
store_nbr family     date                                                     
1         AUTOM

# Sklearn Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error as msle
from tqdm import tqdm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor

import optuna

In [25]:
logs = []

def objective(trial):
    params = {
        'criterion': 'squared_error',
        'bootstrap': trial.suggest_categorical('bootstrap',['True','False']),
        'max_depth': trial.suggest_int('max_depth', 1, 10000),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt','log2']),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 1, 10000),
        'n_estimators': trial.suggest_int('n_estimators', 30, 1000),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20)
    }
    weights_distribution = trial.suggest_categorical('weights_distribution', [1, 2, 3, 4, 5])
    
    sm_rf = 0
    cnt = 0
    
    i = df.index.get_level_values(0).unique()[0]
    for j in df.index.get_level_values(1).unique():
        df_ = df.loc[(i, j)]
        df_ = df_.drop(columns=['id', 'transactions'])

        train = df_[~df_['sales'].isna()]
        X_test = df_[df_['sales'].isna()].drop(columns=['sales'])
        X_train, X_val, y_train, y_val = my_split_func(train)

        y_train = np.log1p(y_train)
        y_val = np.log1p(y_val)
        
        model = RandomForestRegressor(**params)
        weights = get_weights_distribution(weights_distribution, X_train.index)
        model.fit(X_train, y_train, sample_weight=weights)
        preds = model.predict(X_val)
        
        sm_rf += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        cnt += 1

    logs.append([(sm_rf / cnt)**0.5, params, weights_distribution])
    return (sm_rf / cnt)**0.5

In [26]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=1000)

In [27]:
good_p = [{'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9733,
  'max_features': 'auto',
  'max_leaf_nodes': 4730,
  'n_estimators': 159,
  'min_samples_split': 3,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 3637,
  'max_features': 'auto',
  'max_leaf_nodes': 5844,
  'n_estimators': 927,
  'min_samples_split': 19,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5062,
  'max_features': 'auto',
  'max_leaf_nodes': 5517,
  'n_estimators': 802,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 3464,
  'max_features': 'auto',
  'max_leaf_nodes': 6557,
  'n_estimators': 831,
  'min_samples_split': 18,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9643,
  'max_features': 'auto',
  'max_leaf_nodes': 3637,
  'n_estimators': 602,
  'min_samples_split': 8,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5776,
  'max_features': 'auto',
  'max_leaf_nodes': 5125,
  'n_estimators': 850,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4385,
  'max_features': 'auto',
  'max_leaf_nodes': 5692,
  'n_estimators': 622,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9517,
  'max_features': 'auto',
  'max_leaf_nodes': 5239,
  'n_estimators': 154,
  'min_samples_split': 3,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9974,
  'max_features': 'auto',
  'max_leaf_nodes': 1802,
  'n_estimators': 286,
  'min_samples_split': 8,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8942,
  'max_features': 'auto',
  'max_leaf_nodes': 3263,
  'n_estimators': 356,
  'min_samples_split': 14,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5629,
  'max_features': 'auto',
  'max_leaf_nodes': 4907,
  'n_estimators': 586,
  'min_samples_split': 20,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5292,
  'max_features': 'auto',
  'max_leaf_nodes': 5194,
  'n_estimators': 885,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 3123,
  'max_features': 'auto',
  'max_leaf_nodes': 5206,
  'n_estimators': 814,
  'min_samples_split': 7,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4363,
  'max_features': 'auto',
  'max_leaf_nodes': 5692,
  'n_estimators': 816,
  'min_samples_split': 19,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5159,
  'max_features': 'auto',
  'max_leaf_nodes': 5510,
  'n_estimators': 863,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9640,
  'max_features': 'auto',
  'max_leaf_nodes': 3074,
  'n_estimators': 631,
  'min_samples_split': 2,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 3227,
  'max_features': 'auto',
  'max_leaf_nodes': 6524,
  'n_estimators': 931,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5503,
  'max_features': 'auto',
  'max_leaf_nodes': 5540,
  'n_estimators': 813,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 2388,
  'max_features': 'auto',
  'max_leaf_nodes': 5413,
  'n_estimators': 842,
  'min_samples_split': 6,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9395,
  'max_features': 'auto',
  'max_leaf_nodes': 2736,
  'n_estimators': 549,
  'min_samples_split': 3,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5346,
  'max_features': 'auto',
  'max_leaf_nodes': 4506,
  'n_estimators': 600,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5633,
  'max_features': 'auto',
  'max_leaf_nodes': 5546,
  'n_estimators': 802,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9638,
  'max_features': 'auto',
  'max_leaf_nodes': 3230,
  'n_estimators': 347,
  'min_samples_split': 10,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8900,
  'max_features': 'auto',
  'max_leaf_nodes': 3277,
  'n_estimators': 604,
  'min_samples_split': 12,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9684,
  'max_features': 'auto',
  'max_leaf_nodes': 6188,
  'n_estimators': 151,
  'min_samples_split': 3,
  'min_samples_leaf': 7},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4280,
  'max_features': 'auto',
  'max_leaf_nodes': 5767,
  'n_estimators': 582,
  'min_samples_split': 19,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5492,
  'max_features': 'auto',
  'max_leaf_nodes': 7085,
  'n_estimators': 928,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9442,
  'max_features': 'auto',
  'max_leaf_nodes': 3015,
  'n_estimators': 571,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9229,
  'max_features': 'auto',
  'max_leaf_nodes': 2628,
  'n_estimators': 724,
  'min_samples_split': 14,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8943,
  'max_features': 'auto',
  'max_leaf_nodes': 3268,
  'n_estimators': 552,
  'min_samples_split': 12,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5048,
  'max_features': 'auto',
  'max_leaf_nodes': 5318,
  'n_estimators': 841,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4811,
  'max_features': 'auto',
  'max_leaf_nodes': 4645,
  'n_estimators': 862,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9948,
  'max_features': 'auto',
  'max_leaf_nodes': 2266,
  'n_estimators': 478,
  'min_samples_split': 2,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9082,
  'max_features': 'auto',
  'max_leaf_nodes': 2578,
  'n_estimators': 771,
  'min_samples_split': 16,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9473,
  'max_features': 'auto',
  'max_leaf_nodes': 4595,
  'n_estimators': 601,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 2596,
  'max_features': 'auto',
  'max_leaf_nodes': 5962,
  'n_estimators': 826,
  'min_samples_split': 19,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 7853,
  'max_features': 'auto',
  'max_leaf_nodes': 3855,
  'n_estimators': 540,
  'min_samples_split': 12,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9124,
  'max_features': 'auto',
  'max_leaf_nodes': 3312,
  'n_estimators': 561,
  'min_samples_split': 12,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5893,
  'max_features': 'auto',
  'max_leaf_nodes': 7028,
  'n_estimators': 900,
  'min_samples_split': 18,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4892,
  'max_features': 'auto',
  'max_leaf_nodes': 5583,
  'n_estimators': 817,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8662,
  'max_features': 'auto',
  'max_leaf_nodes': 2555,
  'n_estimators': 603,
  'min_samples_split': 2,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9399,
  'max_features': 'auto',
  'max_leaf_nodes': 5406,
  'n_estimators': 476,
  'min_samples_split': 8,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5676,
  'max_features': 'auto',
  'max_leaf_nodes': 5010,
  'n_estimators': 910,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8670,
  'max_features': 'auto',
  'max_leaf_nodes': 2466,
  'n_estimators': 615,
  'min_samples_split': 2,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9504,
  'max_features': 'auto',
  'max_leaf_nodes': 4756,
  'n_estimators': 174,
  'min_samples_split': 3,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5425,
  'max_features': 'auto',
  'max_leaf_nodes': 4880,
  'n_estimators': 593,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 3110,
  'max_features': 'auto',
  'max_leaf_nodes': 6680,
  'n_estimators': 900,
  'min_samples_split': 18,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 7487,
  'max_features': 'auto',
  'max_leaf_nodes': 2261,
  'n_estimators': 619,
  'min_samples_split': 2,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9159,
  'max_features': 'auto',
  'max_leaf_nodes': 1792,
  'n_estimators': 650,
  'min_samples_split': 2,
  'min_samples_leaf': 7},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5681,
  'max_features': 'auto',
  'max_leaf_nodes': 5128,
  'n_estimators': 865,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8890,
  'max_features': 'auto',
  'max_leaf_nodes': 4136,
  'n_estimators': 652,
  'min_samples_split': 2,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5616,
  'max_features': 'auto',
  'max_leaf_nodes': 5129,
  'n_estimators': 805,
  'min_samples_split': 19,
  'min_samples_leaf': 7},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9610,
  'max_features': 'auto',
  'max_leaf_nodes': 3832,
  'n_estimators': 537,
  'min_samples_split': 3,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 7895,
  'max_features': 'auto',
  'max_leaf_nodes': 2550,
  'n_estimators': 547,
  'min_samples_split': 7,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9542,
  'max_features': 'auto',
  'max_leaf_nodes': 4330,
  'n_estimators': 138,
  'min_samples_split': 12,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9248,
  'max_features': 'auto',
  'max_leaf_nodes': 3167,
  'n_estimators': 725,
  'min_samples_split': 9,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9384,
  'max_features': 'auto',
  'max_leaf_nodes': 2723,
  'n_estimators': 542,
  'min_samples_split': 8,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9690,
  'max_features': 'auto',
  'max_leaf_nodes': 3295,
  'n_estimators': 580,
  'min_samples_split': 14,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9740,
  'max_features': 'auto',
  'max_leaf_nodes': 2422,
  'n_estimators': 226,
  'min_samples_split': 12,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 5135,
  'max_features': 'auto',
  'max_leaf_nodes': 4734,
  'n_estimators': 801,
  'min_samples_split': 20,
  'min_samples_leaf': 8},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 7858,
  'max_features': 'auto',
  'max_leaf_nodes': 1368,
  'n_estimators': 588,
  'min_samples_split': 4,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 4094,
  'max_features': 'auto',
  'max_leaf_nodes': 5658,
  'n_estimators': 605,
  'min_samples_split': 20,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 8659,
  'max_features': 'auto',
  'max_leaf_nodes': 2678,
  'n_estimators': 669,
  'min_samples_split': 3,
  'min_samples_leaf': 9},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 7750,
  'max_features': 'auto',
  'max_leaf_nodes': 3738,
  'n_estimators': 703,
  'min_samples_split': 13,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9805,
  'max_features': 'auto',
  'max_leaf_nodes': 3020,
  'n_estimators': 716,
  'min_samples_split': 14,
  'min_samples_leaf': 10},
 {'criterion': 'squared_error',
  'bootstrap': 'False',
  'max_depth': 9567,
  'max_features': 'auto',
  'max_leaf_nodes': 2948,
  'n_estimators': 223,
  'min_samples_split': 12,
  'min_samples_leaf': 10}]

In [28]:
ss = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

In [29]:
sm_rf = 0
cnt = 0
fin_pred = {}
for i in tqdm(df.index.get_level_values(0).unique()):
    for j in df.index.get_level_values(1).unique():
        df_ = df.loc[(i, j)]
        test_id = df_[df_['sales'].isna()]['id']
        df_ = df_.drop(columns=['id', 'transactions'])

        train = df_[~df_['sales'].isna()]
        X_test = df_[df_['sales'].isna()].drop(columns=['sales'])
        X_train, X_val, y_train, y_val = my_split_func(train)

        y_train = np.log1p(y_train)
        # y_val = np.log1p(y_val)

        model = RandomForestRegressor(**good_p[0], random_state=0)
        weights = get_weights_distribution(5, X_train.index)
        model.fit(X_train, y_train, sample_weight=weights)

        # preds = model.predict(X_val)
        preds_ = model.predict(X_test)

        for q in range(preds_.shape[0]):
            fin_pred[test_id[q]] = preds_[q]
        
        # sm_rf += msle(np.exp(y_val) - 1, (np.exp(pr) - 1).clip(0))
        # cnt += 1
#     print((sm_rf / cnt)**0.5)
# print((sm_rf / cnt)**0.5)

100%|██████████| 54/54 [48:42<00:00, 54.12s/it]


In [30]:
# sm_rf = 0
# sm_lg = 0
# cnt = 0

# i = df.index.get_level_values(0).unique()[0]
# for j in df.index.get_level_values(1).unique():
#     df_ = df.loc[(i, j)]
#     df_ = df_.drop(columns=['id', 'transactions'])

#     train = df_[~df_['sales'].isna()]
#     X_test = df_[df_['sales'].isna()].drop(columns=['sales'])
#     X_train, X_val, y_train, y_val = my_split_func(train)

#     y_train = np.log1p(y_train)
#     y_val = np.log1p(y_val)
#     y_test = np.log1p(y_test)
#     weights = np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(X_train.index)).days) / 200)
#     weights = weights.to_numpy()

#     model = RandomForestRegressor()
#     model.fit(X_train, y_train, sample_weight=weights)
#     preds = model.predict(X_val)
#     sm_rf += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))

#     model = LGBMRegressor()
#     model.fit(X_train.to_numpy(), y_train, sample_weight=weights)
#     preds = model.predict(X_val)
#     sm_lg += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))

#     cnt += 1
# print("RF:", (sm_rf / cnt)**0.5)
# print("LG:", (sm_lg / cnt)**0.5)

In [31]:
# sm_gb = 0
# sm_lr = 0
# sm_rf = 0
# sm_xg = 0
# sm_lg = 0
# sm_cb = 0

# cnt = 0
# for i in tqdm(df.index.get_level_values(0).unique()):
#     for j in df.index.get_level_values(1).unique():
#         df_ = df.loc[(i, j)]
#         df_ = df_.drop(columns=['id', 'transactions'])
        
#         train = df_[~df_['sales'].isna()]
#         X_test = df_[df_['sales'].isna()].drop(columns=['sales'])
#         X_train, X_val, y_train, y_val = my_split_func(train)
        
#         y_train = np.log1p(y_train)
#         y_val = np.log1p(y_val)
#         y_test = np.log1p(y_test)
#         weights = np.exp((400 - (pd.to_datetime('2017-08-16') - pd.to_datetime(X_train.index)).days) / 200)
#         weights = weights.to_numpy()
        
#         model = GradientBoostingRegressor()
#         model.fit(X_train, y_train, sample_weight=weights)
#         preds = model.predict(X_val)
#         sm_gb += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
#         model = LinearRegression()
#         model.fit(X_train, y_train, sample_weight=weights)
#         preds = model.predict(X_val)
#         sm_lr += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
#         model = RandomForestRegressor()
#         model.fit(X_train, y_train, sample_weight=weights)
#         preds = model.predict(X_val)
#         sm_rf += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
#         model = XGBRegressor()
#         model.fit(X_train, y_train, sample_weight=weights)
#         preds = model.predict(X_val)
#         sm_xg += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
#         model = LGBMRegressor()
#         model.fit(X_train.to_numpy(), y_train, sample_weight=weights)
#         preds = model.predict(X_val)
#         sm_lg += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
        
#         P = Pool(X_train.to_numpy(), y_train, weight=weights)
#         if len(set(y_train)) == 1:
#             preds = np.full((X_val.shape[0], 1), y_train[0])
#         else:
#             model = CatBoostRegressor()
#             model.fit(X_train, y_train, verbose=0)
#             preds = model.predict(X_val)
#         sm_cb += msle(np.exp(y_val) - 1, (np.exp(preds) - 1).clip(0))
        
#         cnt += 1
#         #print(msle(np.exp(y_val) - 1,(np.exp(preds) - 1).clip(0), squared=False))
#     print("GB:", (sm_gb / cnt)**0.5)
#     print("RF:", (sm_rf / cnt)**0.5)
#     print("LR:", (sm_lr / cnt)**0.5)
#     print("XG:", (sm_xg / cnt)**0.5)
#     print("LG:", (sm_lg / cnt)**0.5)
#     print("CB:", (sm_cb / cnt)**0.5)
#     break
# #print((sm / cnt)**0.5)

# Submission

In [32]:
ss['sales'] = ss['id'].map(fin_pred)

In [33]:
ss['sales'] = np.exp(ss['sales']) - 1

In [34]:
ss.to_csv('submission.csv', index=False)

In [35]:
ss

,id,sales
0,3000888,3.218919
1,3000889,0.000000
2,3000890,4.031745
3,3000891,2420.409059
4,3000892,0.115015
...,...,...
28507,3029395,340.372360
28508,3029396,113.848349
28509,3029397,1272.751190
28510,3029398,116.628746
